In [1]:
# import libraries
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

# DEFINE FUNCTIONS


# change all "off" flow values to zero
def low_flow(flow):
    if flow < 10:
        
        flow = 0
    return flow

# change all "on" flow values to two (for operating minutes)
def operating_minutes(flow):
    if flow > 0:
        
        flow = 2
        
    else:
        
        flow = 0
    return flow

# push dates out of index, cut data down to list of timedelta time values
def make_oplist(dataframe):
    dataframe.reset_index(inplace = True)
    
    df_array = dataframe.to_numpy()
    
    date_time_min = df_array[:, ['Timestamp',6]]
    
    oplist = date_time_min.tolist()
    return oplist

# return list of sublists containing all reads with low flow ("off")
def pull_downtimes(op_list, x): 
    return [item for item in op_list if x in item]

# return list of sublists containing only time element (pass in list 
# with elements [date , time, operating min])
def Extract(flaretime):
    return [item[1] for item in flaretime]

# extracts str time value from downtime list and iterates to make list 
# of datetime time values
def time_type(downtime):
    
    timelist = Extract(downtime)
    
    time_format = [pd.to_datetime(timelist[i]) for i
                   in range(len(downtime))]
        
    return time_format

# iterates through datetime list to create list of timedelta values
# of time difference between reads
def time_difference(list_time):
    for i in range(len(list_time)):
        return [list_time[i]-list_time[i+1] for list_time[i]
                in list_time if list_time]

# make boolean list if difference greater than two mins
def flag_timejumps(list_timediff):
    twomins = 120
    
    list_time_correction = [(i-2) * twomins for i in range(len(list_timediff))]
    
    list_secondsdiff = [list_timediff[i].total_seconds() for i in range(len(list_timediff))]
    
    list_timediff = [list_secondsdiff[i] - list_time_correction[i] for i in range(len(list_secondsdiff))]
    
    check_gap = [list_timediff[i+1] - list_timediff[i] for i in range(len(list_secondsdiff)-1)]
        
    for i in range(len(check_gap)):
        if check_gap[i] > 0:
            check_gap[i] = 1
        else:
            check_gap[i] = 0
    return check_gap

# make list labeling each downtime with integers 1-n based on time jump bool list
def downtime_jump(list_timejump):
    a = 0
    
    downtime_jump = [a for i in range(len(list_timejump))]
    
    n = 1
    
    for i in range(len(downtime_jump)):
        
        if list_timejump[i-1] == 1:
            
            n = n + 1
            
        downtime_jump[i] = downtime_jump[i] + n
    return downtime_jump
